Generar un mapa de áreas para indicar los lugares con cierta cantidad de tweets y ponerlos en rango de color (reporte)

Generar el random point en las carreteras, o mejorer el proceso hacia el diagrama planteado.

Delimitar el área de las carreteras solo a la ciudad de Panamá

In [1]:
import psycopg2 as ps
from wordcloud import WordCloud
import  matplotlib.pyplot as plt 
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px

import re, sys
import numpy as np

sys.path.append('./TweetData')
from TweetData import keys, Preprocessing, Detection, DBConnect

import pandas as pd
import json
from collections import Counter
from fuzzywuzzy import fuzz, process

import geopandas as gpd

In [2]:
from Detection import rf_classifier, vectorizer3

In [3]:
db = DBConnect.DB_Connection()
db.connect(password=keys.db_pass)

True

## Misc

In [9]:
places1 = places.copy()
places1.geom = places1.geom.to_crs(epsg=4326)

In [20]:
def generate_scatter(df, geom, info):
    fig = go.Figure(
        go.Scattermapbox(
            lat=df[geom].y, lon=df[geom].x,
            marker=go.scattermapbox.Marker(
                size=10,
                color='red',
                opacity=0.7
            ),
            text=df[info],
            hoverinfo=['text']
        )
    )
    fig.update_geos(
        resolution=110,
    )
    fig.update_layout(margin={"r":0,"t":50,"l":0,"b":0},
        mapbox_style="open-street-map", mapbox_zoom=11,
        mapbox_center={'lat':8.99, 'lon':-79.5359},
        title='Mapa de lugares',
        hoverlabel=dict(
            bgcolor="white",
            font_size=12,
        )
    )
    return fig

## Places queries

In [4]:
## Get all tweets no places
conn = ps.connect(
    database='traffictwt', user='postgres', password='root')
cursor = conn.cursor()
q = '''SELECT TWEET_ID, TWEET_TEXT FROM TWEETS ORDER BY TWEET_CREATED DESC LIMIT 50'''
cursor.execute(q)
tweets = cursor.fetchall()
cursor.close()
conn.close()

tweets = pd.DataFrame(tweets, columns=['id', 'text'])

In [5]:
# Get list of places
conn = ps.connect(
    database='traffictwt', user='postgres', password='root')
q = '''SELECT OSM_ID, NAME, ST_CENTROID(way) as geom FROM PLACES '''
places = gpd.GeoDataFrame.from_postgis(q, conn, geom_col='geom')

In [6]:
# Get list of roads
conn = ps.connect(
    database='traffictwt', user='postgres', password='root')
q1 = '''SELECT GID, NOMBRE, GEOM FROM carreteras WHERE NOMBRE IS NOT NULL '''
roads = gpd.GeoDataFrame.from_postgis(q1, conn, geom_col='geom')

### Get tweets with places assigned

In [81]:
# Get tweets roads
conn = ps.connect(
    database='traffictwt', user='postgres', password='root')
q1 = '''SELECT TP.TWEET_ID, T.TWEET_TEXT, R.NOMBRE AS ROAD_NAME, ST_CENTROID(R.GEOM) AS ROAD_GEOM
    FROM TWEETS_PLACES AS TP
        INNER JOIN CARRETERAS AS R ON TP.ROAD_GID=R.GID 
        INNER JOIN TWEETS AS T ON TP.TWEET_ID=T.TWEET_ID '''
# TWEETS_PLACES(TWEET_ID, ROAD_GID, PLACE_ID)
twt_roads = gpd.GeoDataFrame.from_postgis(q1, conn, geom_col='road_geom')

In [82]:
# Get tweets places
conn = ps.connect(
    database='traffictwt', user='postgres', password='root')
q1 = '''SELECT TP.TWEET_ID, T.TWEET_TEXT, P.NAME AS PLACE_NAME, ST_CENTROID(P.WAY) as PLACE_GEOM
    FROM TWEETS_PLACES AS TP
        INNER JOIN TWEETS AS T ON TP.TWEET_ID=T.TWEET_ID 
        INNER JOIN PLACES AS P ON TP.PLACE_ID=P.OSM_ID '''
# TWEETS_PLACES(TWEET_ID, ROAD_GID, PLACE_ID)
twt_places = gpd.GeoDataFrame.from_postgis(q1, conn, geom_col='place_geom')
twt_places.place_geom = twt_places.place_geom.to_crs(epsg=4326)

In [84]:
fig = go.Figure(
    [go.Scattermapbox(
        lat=twt_roads['road_geom'].y, 
        lon=twt_roads['road_geom'].x,
        marker=go.scattermapbox.Marker(
            size=10,
            color='red',
            opacity=0.7
        ),
        text=twt_roads['tweet_text'].str.wrap(30).apply(lambda x: 
            x.replace('\n', '<br>')),
        hoverinfo=['text'],
        hovertemplate='%{text}<extra></extra>'
    # ),
    # go.Scattermapbox(
    #     lat=twt_places['place_geom'].y, 
    #     lon=twt_places['place_geom'].x,
    #     marker=go.scattermapbox.Marker(
    #         size=10,
    #         color='blue',
    #         opacity=0.7
    #     ),
    #     text=twt_places['tweet_text'].str.wrap(30).apply(lambda x: 
    #         x.replace('\n', '<br>')),
    #     hoverinfo=['text'],
    #     hovertemplate='%{text}<extra></extra>'
    )]
)
fig.update_geos(
    resolution=110,
)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},
    mapbox_style="open-street-map", mapbox_zoom=11.5,
    mapbox_center={'lat':8.98, 'lon':-79.5359},
    hoverlabel=dict(
        bgcolor="white",
        font_size=12,
    )
)
fig.show()

In [67]:
## Merge tweets to get roads and places in one dataframe
twt_roads.merge(twt_places, how='outer', on=['tweet_id', 'tweet_text'])

,tweet_id,tweet_text,road_name,road_geom,place_name,place_geom
0,1401320841711923200,[VÍDEO] Se registra colisión en la Avenida Ric...,Avenida Ricardo J. Alfaro,POINT (-79.53378 8.99212),NaN,None
1,1401277242362281985,"Vistas del tráfico en la Avenida Domingo Díaz,...",Avenida A,POINT (-79.54603 8.95116),San Antonio,POINT (-79.44505 9.05188)
2,1401256978396192770,"Trabajos en la Cinta Costera, mantiene el tráf...",Cinta Costera,POINT (-79.53999 8.95783),Cinta Costera 1,POINT (-79.52108 8.97516)
3,1401255933825671170,[VÍDEO] Se registra colisión en la Carretera P...,Carretera Panamericana,POINT (-80.56366 8.24666),Panamericana,POINT (-79.52278 8.98787)
4,1401232228118634499,"[VÍDEO] Trabajos en la Cinta Costera, mantiene...",Cinta Costera,POINT (-79.53999 8.95783),Cinta Costera 1,POINT (-79.52108 8.97516)
...,...,...,...,...,...,...
434,1397259083263193093,"Protesta en la vía Transístmica, frente al Com...",NaN,None,San Miguelito,POINT (-79.50659 9.03002)
435,1397258426686791680,Reportan inundaciones en la barriada Plaza Val...,NaN,None,Plaza Villa Zaita,POINT (-79.51669 9.08132)
436,1397257812842065924,Reportan inundadaciones en la Barriada Plaza V...,NaN,None,Plaza Villa Zaita,POINT (-79.51669 9.08132)
437,1397256955857625089,"[VÍDEO] Protesta en la Vía Transístmica, próxi...",NaN,None,Carga Transistmica,POINT (-79.37505 9.08259)


Tomar la vía 
-> si detecta la vía, tomar lugares adyacentes a la vía, luego hacer un fuzzy match a los lugares cercanos y asignar la intersección

Si no detecta una vía, hacer fuzzy match al texto por un lugares

Si no, asignar nulo

### Create X and Y coordinates for places

In [ ]:
places1.loc[:, 'x'] = places1.apply(lambda row: getPointCoords(row, 'geom', 'x'), axis=1)
places1.loc[:, 'y'] = places1.apply(lambda row: getPointCoords(row, 'geom', 'y'), axis=1)

In [ ]:
def getPointCoords(row, geom, coord_type):
    """Calculates coordinates ('x' or 'y') of a Point geometry"""
    if coord_type == 'x':
        return row[geom].x
    elif coord_type == 'y':
        return row[geom].y

In [68]:
places = places[places.name.str.len()>5]

## Extract Places

In [69]:
extract = Preprocessing.clean_text(tweets, 'text')
# tweets.clean.str.findall('([A-Z]\S+)')
# re.findall('([A-Z]\S+)', tweets)
extract['possible_place'] = extract.clean.str.findall('([A-Z]\S+)')

In [70]:
extract = extract[extract.clean.str.contains('accident.*|cho[qc].*|colision.*|vuelco|volcó|vuelca|atropell.*|tran[qc].*|embotella.*|trafico|huelga|protest.*|trabajo.*|cierr.*|detenid.*|obstacul.*|incendi.*|inunda.*|lluvia')]
extract = extract[extract.clean.str.len()>30]
extract = extract[extract.possible_place.notnull()]

### Fuzzy match tests

In [64]:
## Fuzzy match tests
test = 'tranque pesado Cinta Costera area Coco Mar'

In [68]:
process.extract(test, places['name'], scorer=fuzz.token_set_ratio)

[('Cinta Costera 1', 93, 5358),
 ('Cinta Costera 1', 93, 5404),
 ('Cinta Costera 3', 93, 6432),
 ('Cinta Costera 3', 93, 6435),
 ('Cinta Costera 3', 93, 6450)]

In [66]:
set_ratio = process.extractOne(test, places['name'], scorer=fuzz.token_set_ratio)
set_ratio

('Cinta Costera 1', 93, 5358)

In [ ]:
extract['contains'] = extract['clean'].apply(lambda x: np.array([]))
# extract['contains'] = extract['contains'].astype(object)
extract['possible'] = extract['clean'].apply(lambda x: np.array([]))
# extract['possible'] = extract['possible'].astype(object)

In [60]:
extract.drop(columns=['contains', 'possible'], inplace=True)

In [ ]:
extract2 = extract.copy()

In [71]:
def get_road(text, roads, scorer=fuzz.token_set_ratio):
    '''Applies fuzzy match to extract the possible road of the text.'''
    choice = process.extractOne(text, roads.nombre, scorer=scorer)
    if choice[1]>=80:
        # Returns the gid
        return str(roads.iloc[roads.index==choice[2], 0].iloc[0])
    else: return ''

In [72]:
def get_place(text, places, scorer=fuzz.token_set_ratio):
    '''Applies fuzzy match to extract the possible road of the text.'''
    choice = process.extractOne(text, places.name, scorer=scorer)
    if choice[1]>=80:
        # Returns the osm_id
        return str(places.iloc[places.index==choice[2], 0].iloc[0])
    else: return ''

In [ ]:
places.head(1)

In [73]:
extract2 = extract[500:1000]

In [77]:
## Tackes an extract and searches the possible road
extract2.loc[:, 'road'] = extract2.apply(lambda row: get_road(row.clean, roads), axis=1)

C:\Users\lucia\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\indexing.py:1597: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\lucia\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
## Takes part of extract and searches possible place
extract2.loc[:, 'place'] = extract2.apply(lambda row: get_place(row.clean, places), axis=1)

In [78]:
## Inserting results into database
conn = ps.connect(
    database='traffictwt', user='postgres', password='root')
cursor = conn.cursor()
ins = '''INSERT INTO TWEETS_PLACES(TWEET_ID, ROAD_GID, PLACE_ID)
    VALUES (%s, %s, %s) '''
for index, row in extract2.iterrows():
    cursor.execute(ins, 
        (row['id'], None if row['road']=='' else row['road'], 
            None if row['place']=='' else row['place']))
    conn.commit()
cursor.close()
conn.close()

In [79]:
extract2

,id,text,clean,possible_place,place,road
762,1397250548118794244,Camión articulado detenido y en medio de una c...,camion articulado detenido medio columna humo ...,"[Via, Centenario, Rod, Carew, Panama, Oeste, V...",385466258,
763,1397248875363250185,Se registra accidente de tránsito en el Corred...,registra accidente transito Corredor Norte Tin...,"[Corredor, Norte, Tinajitas, Panama, Centro, V...",684868242,2162
764,1397243505769164800,[VÍDEO] Se registra accidente de tránsito en e...,registra accidente transito Corredor Norte Tin...,"[Corredor, Norte, Tinajitas, Panama, Centro]",684868242,2162
765,1397243504280145921,"[VÍDEO] Mula detenida en la Vía Centenario, pr...",mula detenida Via Centenario Rod Carew direcci...,"[Via, Centenario, Rod, Carew, Panama, Oeste]",385466258,
766,1397227764277780499,Protesta por calle\n\nResidentes de Gonzalillo...,protesta calle Residentes Gonzalillo salieron ...,"[Residentes, Gonzalillo, Video]",412554328,
...,...,...,...,...,...,...
1571,1391083910914842625,[VÍDEO] Se registra accidente de tránsito en L...,registra accidente transito Via Centenario Rod...,"[Via, Centenario, Rod, Carew, Panama, Oeste]",385466258,
1572,1391082349895507969,[VÍDEO] Se registra accidente en la Vía España...,registra accidente Via España Cresta direccion...,"[Via, España, Cresta, Panama, Centro]",,
1573,1391078584329068552,[VÍDEO] Se registra colisión en la Avenida Ric...,registra colision Avenida Ricardo J Alfaro San...,"[Avenida, Ricardo, Alfaro, San, Miguel, Arcang...",,2192
1574,1391077923894042624,[VÍDEO] Vistas del tráfico en la Vía Interamer...,vistas trafico Via Interamericana Coronado dir...,"[Via, Interamericana, Coronado, Interior, Pais]",384384811,


## More detailed process with intweet and possible place

### Cleaning

In [20]:
# extract1 = extract.copy()
tweets = Detection.get_classifications(tweets, 'text')

In [26]:
tweets['contains'] = tweets['clean'].apply(lambda x: np.array([]))
tweets['contains'] = tweets['contains'].astype(object)
tweets['possible'] = tweets['clean'].apply(lambda x: np.array([]))
tweets['possible'] = tweets['possible'].astype(object)

In [40]:
tweets['text']

0                                            😂🤣😂😂🤣😂🤣🤣😂🤣
1     [VIDEO] Vistas del tráfico en la vía Interamer...
2     [VIDEO] Accidente de tránsito en Penonomé, Coc...
3     Gracias a las acciones realizadas el 5 de novi...
4     La ATTT hace un llamado a gremios, sindicatos,...
5     [VIDEO] A través de las cámaras de video vigil...
6     A 5km/h se desplazan los autos en la Vía Inter...
7     [VIDEO] Accidente de tránsito en la vía Intera...
8     Vistas del tráfico en la vía Interamericana en...
9     [VIDEO] A través de las cámaras de vigilancia ...
10    El subcomisionado Fernando Sanjur de la DNOT s...
11    [VIDEO] Un vehículo tipo sedán, cuyo conductor...
12    [VIDEO] Más vistas del vuelco en el Kilómetro ...
13    Desde la provincia de Colón se reporta acciden...
14    Feliz viernes a todos nuestros seguidores exce...
15    Zona paga de Torrijos Carter. Han llegado cinc...
16    Árbol cae en la ví hacia el Valle de Antón, se...
17    @ProtegeryServir @PGN_PANAMA @panamamunici

In [50]:
tweets.iloc[21, 2]

'10kmh desplazan autos Via Interamericana Coronado direccion Panama Centro '

In [57]:
intweet = []; possible = []

test = tweets.iloc[21, 2]
for index_place, place in places.iterrows():
    if place['name'] in test:
        # Insert the ratio and place ID
        intweet.append([fuzz.token_sort_ratio(place['name'], test), place['osm_id']])
        print(fuzz.token_sort_ratio(place['name'], test), place['name'])
    elif (fuzz.token_set_ratio(place['name'], test)==100 
        and fuzz.token_sort_ratio(place['name'], test)>50) or (fuzz.token_set_ratio(place['name'], test)>90):
        possible.append([fuzz.token_sort_ratio(place['name'], test), place['osm_id']])


8 Cor
13 plaza
13 Panam


In [69]:
## Searches if a place is exactly in the text or a possible match
# for index, tweet in extract[:100].iterrows():
for index, tweet in tweets.iterrows():
    intweet = []; possible = []

    ## Iteration of the places in the database
    for index_place, place in places.iterrows():
        if len(place['name'])>5:
            if place['name'] in tweet['clean']:
                # Insert the ratio and place ID
                intweet.append([fuzz.token_sort_ratio(place['name'], tweets.iloc[12, 2]), place['osm_id']])
            elif (fuzz.token_set_ratio(place['name'], tweet['clean'])==100 
                and fuzz.token_sort_ratio(place['name'], tweet['clean'])>50) or (fuzz.token_set_ratio(place['name'], tweet['clean'])>90):
                possible.append([fuzz.token_sort_ratio(place['name'], tweets.iloc[12, 2]), place['osm_id']])
    if len(intweet)>0:
        tweets.loc[index, 'contains_place'] = max(intweet)[1]
    if len(possible)>0:
        tweets.loc[index, 'possible_place'] = max(possible)[1] # np.array(possible)

    ## Same process with roads
    intweet = []; possible = []
    ## Iteration of roads
    for index_road, road in roads.iterrows():
        if len(road['nombre'])>5:
            if road['nombre'] in tweet['clean']:
                # Insert the ratio and road ID
                intweet.append([fuzz.token_sort_ratio(road['nombre'], tweets.iloc[12, 2]), road['gid']])
            elif (fuzz.token_set_ratio(road['nombre'], tweet['clean'])==100 
                and fuzz.token_sort_ratio(road['nombre'], tweet['clean'])>50) or (fuzz.token_set_ratio(road['nombre'], tweet['clean'])>90):
                possible.append([fuzz.token_sort_ratio(road['nombre'], tweets.iloc[12, 2]), road['gid']])
    if len(intweet)>0:
        tweets.loc[index, 'contains_road'] = max(intweet)[1]
    if len(possible)>0:
        tweets.loc[index, 'possible_road'] = max(possible)[1] # np.array(possible)
    

KeyboardInterrupt: 

### DAtabase

In [98]:
tweet = tweets.iloc[12, 1]
tweet

'[VIDEO] Más vistas del vuelco en el Kilómetro 47 de la autopista Panamá-Colón en la salida a Sabanitas. https://t.co/DL7m8qjbNK'

In [99]:
tweet = tweets.iloc[12, 1]
geo = Detection.get_geo(tweet, places, roads)
geo

{'tweet': '[VIDEO] Más vistas del vuelco en el Kilómetro 47 de la autopista Panamá-Colón en la salida a Sabanitas. https://t.co/DL7m8qjbNK', 'place': None, 'road': 2536}


In [100]:
roads[roads.gid==2536]

,gid,nombre,geom
2538,2536,Autopista Panamá - Colón,"MULTILINESTRING ((-79.74012 9.28473, -79.73890..."


In [23]:
geo = {}

intweet = []; possible = []

## Iteration of the places in the database
for index_place, place in places.iterrows():
    if len(place['name'])>5:
        if place['name'] in tweet:
            # Insert the ratio and place ID
            intweet.append([fuzz.token_sort_ratio(place['name'], tweets), place['osm_id']])
        elif (fuzz.token_set_ratio(place['name'], tweet)==100 
            and fuzz.token_sort_ratio(place['name'], tweet)>50) or (fuzz.token_set_ratio(place['name'], tweet)>90):
            possible.append([fuzz.token_sort_ratio(place['name'], tweets), place['osm_id']])
if len(intweet)>0:
    #tweets.loc[index, 'contains_place'] = max(intweet)[1]
    geo['place'] = max(intweet)[1]
elif len(possible)>0:
    # tweets.loc[index, 'possible_place'] = max(possible)[1] # np.array(possible)
    geo['place'] = max(possible)[1]

## Same process with roads
intweet = []; possible = []
## Iteration of roads
for index_road, road in roads.iterrows():
    if len(road['nombre'])>5:
        if road['nombre'] in tweet:
            # Insert the ratio and road ID
            intweet.append([fuzz.token_sort_ratio(road['nombre'], tweet), road['gid']])
        elif (fuzz.token_set_ratio(road['nombre'], tweet)==100 
            and fuzz.token_sort_ratio(road['nombre'], tweet)>50) or (fuzz.token_set_ratio(road['nombre'], tweet)>90):
            possible.append([fuzz.token_sort_ratio(road['nombre'], tweet), road['gid']])
if len(intweet)>0:
    # tweets.loc[index, 'contains_road'] = max(intweet)[1]
    geo['road'] = max(intweet)[1]
elif len(possible)>0:
    # tweets.loc[index, 'possible_road'] = max(possible)[1] # np.array(possible)
    geo['road'] = max(possible)[1]

In [7]:
tweets.iloc[1:2].id

1    1453504659671420931
Name: id, dtype: int64

In [12]:
## Inserting results into database
conn = ps.connect(
    database='traffictwt', user='postgres', password='root')
cursor = conn.cursor()
ins = '''INSERT INTO TWEETS_PLACES(TWEET_ID, ROAD_GID, PLACE_ID)
    VALUES (%s, %s, %s) '''
# for index, row in extract2.iterrows():
cursor.execute(ins, 
    (1453504659671420931, geo['place'], geo['road']))
conn.commit()
cursor.close()
conn.close()

In [10]:
db.assign_place(1453504659671420931, geo['place'], geo['road'])

TypeError: exception() missing 1 required positional argument: 'msg'

In [ ]:
## Merge tweets to get roads and places in one dataframe
twt_roads.merge(twt_places, how='outer', on=['tweet_id', 'tweet_text'])